<a href="https://colab.research.google.com/github/FatemehMomeni/BigData2/blob/main/BigData2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 47.4 MB/s 


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install gdown
!pip install --upgrade --no-cache-dir gdown
!gdown --id 1y1BXUZPKDke_LpL7Lmgr4v1WzOrGg1qw
!unzip "gdrive/MyDrive/HW2_dataset.zip" -d "./"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=d1396881a0e48a7770f9a023031cc067f935245568fccb17075ad89b8d23ee81
  Stored in directory: /tmp/pip-ephem-wheel-cache-ft4um4js/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1y1BXUZPKDke_LpL7Lmgr4v1WzOrGg1qw
To: /content/HW2_dataset.zip
100% 6.43G/6.43G [01:28<00:00, 72.5MB/s]
Archive:  gdrive/MyDrive/HW2_dataset.zip
   cre

***Reading Files From Google Drive***

In [5]:
import pyspark
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Spark1").config('spark.ui.port', '4040').getOrCreate()

products = spark.read.parquet("products_parquet")
sales = spark.read.parquet("sales_parquet")
sellers = spark.read.parquet("sellers_parquet")

***Converting Pyspark DataFrame to Pyspark Pandas Dataframe***

In [36]:
import pyspark.pandas as ps

products_pd = products.to_pandas_on_spark()
sales_pd = sales.to_pandas_on_spark()
sellers_pd = sellers.to_pandas_on_spark()

# سوال ۱

## الف

In [7]:
orders_num = ps.sql("select count(order_id) from {sales_pd}")
products_num = ps.sql("select count(product_id) from {products_pd}")
sellers_num = ps.sql("select count(seller_id) from {sellers_pd}")
print("Number of orders:\n"+str(orders_num)+"\n\nNumber of products:\n"+str(products_num)+"\n\nNumber of sellers:\n"+str(sellers_num))

Number of orders:
   count(order_id)
0         20000040

Number of products:
   count(product_id)
0           75000000

Number of sellers:
   count(seller_id)
0                10


In [8]:
orders_num = sales.count()
products_num = products.count()
sellers_num = sellers.count()
print("Number of orders:\n"+str(orders_num)+"\n\nNumber of products:\n"+str(products_num)+"\n\nNumber of sellers:\n"+str(sellers_num))

Number of orders:
20000040

Number of products:
75000000

Number of sellers:
10


## ب

In [9]:
ps.sql("select count(distinct product_id) from {sales_pd}")

,count(DISTINCT product_id)
0,993429


In [10]:
from pyspark.sql.functions import countDistinct
sold_products = sales.select(countDistinct("product_id"))
sold_products.show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                    993429|
+--------------------------+



## ج

In [8]:
#num_sold_pro = ps.sql("select sp.product_id,product_name,count(sp.product_id) as sold_p from {sales_pd} as sp,{products_pd} as pp where sp.product_id=pp.product_id group by sp.product_id,pp.product_name")
num_sold_pro = ps.sql("select sp.product_id,product_name,sum(sp.num_pieces_sold) as sold_p from {sales_pd} as sp,{products_pd} as pp where sp.product_id=pp.product_id group by sp.product_id,pp.product_name")
ps.sql("select product_id,product_name,sold_p as most_frequent from {num_sold_pro} where sold_p=(select max(sold_p) from {num_sold_pro})")

,product_id,product_name,most_frequent
0,0,product_0,959445802.0


In [9]:
#num_sold = sales.groupBy('product_id').count().withColumnRenamed("count","frequency")
num_sold = sales.groupBy('product_id').agg({'num_pieces_sold':'sum'}).withColumnRenamed("sum(num_pieces_sold)","frequency")
num_sold.filter(num_sold.frequency == num_sold.agg({'frequency':'max'}).collect()[0][0]).show()

+----------+------------+
|product_id|   frequency|
+----------+------------+
|         0|9.59445802E8|
+----------+------------+



# سوال ۲

In [20]:
ps.sql("select date,count(distinct product_id) from {sales_pd} group by date")

,date,count(DISTINCT product_id)
0,2020-07-03,100017
1,2020-07-07,99756
2,2020-07-01,100337
3,2020-07-08,99662
4,2020-07-04,99791
5,2020-07-10,98973
6,2020-07-09,100501
7,2020-07-06,100765
8,2020-07-02,99807
9,2020-07-05,99796


In [21]:
sales.select('product_id','date').distinct().groupBy('date').count().show()

+----------+------+
|      date| count|
+----------+------+
|2020-07-03|100017|
|2020-07-07| 99756|
|2020-07-01|100337|
|2020-07-08| 99662|
|2020-07-04| 99791|
|2020-07-10| 98973|
|2020-07-09|100501|
|2020-07-06|100765|
|2020-07-02| 99807|
|2020-07-05| 99796|
+----------+------+



# سوال ۳

In [22]:
ps.sql("select avg(price) from {sales_pd},{products_pd} where {sales_pd}.product_id={products_pd}.product_id")

,avg(price)
0,24.67622


# سوال ۴

In [52]:
sales_sellers = ps.sql("select sa.seller_id,sa.num_pieces_sold*100/se.daily_target as order_percent from {sales_pd} as sa,{sellers_pd} as se where sa.seller_id==se.seller_id")
ps.sql("select seller_id,avg(order_percent) from {sales_sellers} group by seller_id")

,seller_id,avg(order_percent)
0,0,0.002020
1,7,0.002595
2,3,0.016289
3,8,0.009213
4,5,0.004211
5,6,0.004782
6,9,0.003838
7,1,0.019642
8,4,0.003296
9,2,0.006690


# سوال ۵

## الف

In [10]:
num_sold_pro = ps.sql("select sp.seller_id,sep.seller_name,count(sp.seller_id) as sold_p from {sales_pd} as sp,{sellers_pd} as sep where sp.seller_id=sep.seller_id group by sp.seller_id,sep.seller_name")
second_max = ps.sql("select * from(select row_number() over(order by sold_p desc) as row_num,seller_id,seller_name,sold_p as num_sold_products from {num_sold_pro}) as sub where row_num=2")
second_min = ps.sql("select * from(select row_number() over(order by sold_p) as row_num,seller_id,seller_name,sold_p as num_sold_products from {num_sold_pro}) as sub where row_num=2")
print("The Second Best Seller:\n\n" + str(second_max) + "\n\n\nThe Second Worst Seller:\n\n" + str(second_min))

The Second Best Seller:

   row_num seller_id seller_name  num_sold_products
0        2         9    seller_9             111392


The Second Worst Seller:

   row_num seller_id seller_name  num_sold_products
0        2         5    seller_5             110874


## ب

In [31]:
ps.sql("select seller_id from {sales_pd} where product_id=0 group by seller_id")

,seller_id
0,0


In [32]:
sales.filter(sales.product_id=='0').groupBy('seller_id').agg({'num_pieces_sold':'sum'}).show()

+---------+--------------------+
|seller_id|sum(num_pieces_sold)|
+---------+--------------------+
|        0|        9.59445802E8|
+---------+--------------------+



# سوال ۶

In [60]:
from pyspark.sql.functions import sha2,udf,col,lit
from hashlib import md5
from pyspark.sql.types import StringType

@udf
def hashing(order_id,bill_raw_text):
  if order_id % 2 ==0:
    for l in bill_raw_text:
      if l == 'A':
        return md5(bill_raw_text.encode("utf-8") ).hexdigest()
  else:
    for l in bill_raw_text:
      if l == 'A':
        return sha2(bill_raw_text,256)

hashed = udf(lambda x: hashing(x))

sales1 = sales.withColumn("hashed_bill",lit(hashing(sales["order_id"], sales["bill_raw_text"])))
sales2 = sales1.groupBy("hashed_bill").count()